RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [3]:
import io
import os
import numpy as np
import scipy

from sklearn.feature_extraction.text import TfidfVectorizer


In [18]:
PATH_TO_DATA = "data/"

# 1) Monolingual (English) word embeddings 

In [5]:
class Word2vec():
    def __init__(self, fname, nmax=100000):
        self.load_wordvec(fname, nmax)
        self.word2id = dict.fromkeys(self.word2vec.keys())
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.embeddings = np.array(self.word2vec.values())
    
    def load_wordvec(self, fname, nmax):
        self.word2vec = {}
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        print('Loaded %s pretrained word vectors' % (len(self.word2vec)))

    def most_similar(self, w, K=5):
        scores=[self.score(w,w0) for w0 in self.word2vec]
        if max(scores)>0:
            best_words = (np.argsort(scores)[::-1][1:K+1])
            return list(np.array(list(self.word2vec.keys()))[best_words])
        else:
            print('Word not available')
            return 0
#        sim_words=[]
#        list_scores = []
#        list_words=[]
#        for x in list(w2v.word2vec.keys()):
#            list_scores.append(self.score(x,w))
#            list_words.append(x)
#        scores_order= np.argsort(np.array(list_scores))
#        for i in range(K):
#            sim_words.append(list_words[scores_order[i]])
#        return sim_words


            
                      
        # K most similar words: self.score  -  np.argsort 
        return

    def score(self, w1, w2):
        if w1 in list(w2v.word2vec.keys()):
            if w2 in list(w2v.word2vec.keys()):
                cos_sim = np.dot(w2v.word2vec[w1],w2v.word2vec[w2])/(np.linalg.norm(w2v.word2vec[w1])*np.linalg.norm(w2v.word2vec[w2]))
                return cos_sim
        else:
            return 0
        # cosine similarity: np.dot  -  np.linalg.norm
        
        return 


In [6]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=25000)

# You will be evaluated on the output of the following:
for w1, w2 in zip(('cat', 'dog', 'dogs', 'paris', 'germany'), ('dog', 'pet', 'cats', 'france', 'berlin')):
    print(w1, w2, w2v.score(w1, w2))
for w1 in ['cat', 'dog', 'dogs', 'paris', 'germany']:
    print(w2v.most_similar(w1))


Loaded 25000 pretrained word vectors
cat dog 0.671683666279249
dog pet 0.684206402966922
dogs cats 0.7074389328052404
paris france 0
germany berlin 0


KeyboardInterrupt: 

In [15]:
class BoV():
    def __init__(self, w2v):
        self.w2v = w2v
    
    def encode(self, sentences, idf=False):
        # takes a list of sentences, outputs a numpy array of sentence embeddings
        # see TP1 for help
        sentemb = []
        if idf is False:
            # mean of word vectors
            for sent in sentences:
                avail_sent = set(sent).intersection(set(list(w2v.word2vec.keys())))
                if len(avail_sent)>0:
                    mean_vec = sum([self.w2v.word2vec[w] for w in avail_sent])/len(avail_sent)
                    sentemb.append(mean_vec)
                else:
                    sentemb.append(np.zeros(300))
        else:
            # idf-weighted mean of word vectors     
            for sent in sentences:
                avail_sent = set(sent).intersection(set(list(w2v.word2vec.keys())))
                if len(avail_sent) >0:
                    mean_vec = sum([self.w2v.word2vec[w]*idf[w] for w in avail_sent])/len(avail_sent)
                    sentemb.append(mean_vec)
                else:
                    sentemb.append(np.zeros(300))
                    
                
                
        return np.vstack(sentemb)

    def most_similar(self, s, sentences, idf=False, K=5):
        scores = []
        for x in sentences:
            scores.append(self.score(s,x,idf))
            
        best_sentences = np.argsort(scores)[::-1][1:K+1]
        return list(np.array(sentences)[best_sentences])
        
        
        
        # get most similar sentences and **print** them
        #keys = self.encode(sentences, idf)
        #query = self.encode([s], idf)
        

    def score(self, s1, s2, idf=False):
        two_vecs = self.encode([s1,s2],idf)
        cos_sim = np.dot(two_vecs[0],two_vecs[1])/(np.linalg.norm(two_vecs[0])*np.linalg.norm(two_vecs[1]))
        
        
        # cosine similarity: use   np.dot  and  np.linalg.norm
        return cos_sim
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        
        idf = {}
        
        for sent in sentences:
            for w in set(sent):
                idf[w] = idf.get(w, 0) + 1
        
        for w, val in idf.items():
            idf[w]= max(1, np.log10(len(sentences) / (idf[w])))
        
        return idf
        

In [16]:
idf = s2v.build_idf(sentences)



NameError: name 'sentences' is not defined

In [21]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=5000)
s2v = BoV(w2v)

# Load sentences in "PATH_TO_DATA/sentences.txt"
file = open('data/sentences.txt', 'r') 
sentences__not_split=  file.read().splitlines()
sentences = [sent[:-3].split(" ") for sent in sentences__not_split]


# Build idf scores for each word
idf = s2v.build_idf(sentences)

# You will be evaluated on the output of the following:
s2v.most_similar(sentences[10], sentences)  # BoV-mean
s2v.score(sentences[7], sentences[13])



#idf = {}
s2v.most_similar(sentences[10], sentences, idf)  # BoV-idf
s2v.score(sentences[7], sentences[13], idf)

Loaded 5000 pretrained word vectors


0.49886729880784486

# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [45]:
# 1 - Download and load 50k first vectors of
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

# TYPE CODE HERE

PATH_TO_DATA = "data/"

with io.open(os.path.join(PATH_TO_DATA, 'wiki.en.vec'), encoding='utf-8') as f:
    next(f)
    vecs_en= {}
    for x in range(50000):
        line = f.readline()
        word, vec = line.split(' ', 1)
        vecs_en[word] = np.fromstring(vec, sep=' ')
        
with io.open(os.path.join(PATH_TO_DATA, 'wiki.fr.vec'), encoding='utf-8') as f:
    next(f)
    vecs_fr= {}
    for x in range(50000):
        line = f.readline()
        word, vec = line.split(' ', 1)
        vecs_fr[word] = np.fromstring(vec, sep=' ')


In [46]:
# 2 - Get words that appear in both vocabs (= identical character strings)
#     Use it to create the matrix X and Y (of aligned embeddings for these words)

# TYPE CODE HERE
common_words = set(vecs_en.keys()).intersection(set(vecs_fr.keys()))

X = np.matrix([vecs_fr[w] for w in common_words])

Y = np.matrix([vecs_en[w] for w in common_words])



In [63]:
# 3 - Solve the Procrustes using the scipy package and: scipy.linalg.svd() and get the optimal W
#     Now W*French_vector is in the same space as English_vector

# TYPE CODE HERE
U, s, Vh = scipy.linalg.svd(np.dot(X.transpose(),Y))

W = np.dot(U,Vh)

aligned_fr = np.dot(list(vecs_fr.values()),W)

In [86]:
# 4 - After alignment with W, give examples of English nearest neighbors of some French words (and vice versa)
#     You will be evaluated on that part and the code above

# TYPE CODE HERE

fr_aligned = {}
fr_wordlist = list(vecs_fr.keys())
for i in range(len(aligned_fr)):
    fr_aligned[fr_wordlist[i]]=aligned_fr[i]
    
en_d = list(vecs_en.keys())
fr_d = list(vecs_fr.keys())

    
for w in ['chien','chaise']:
    scores =[]
    for word in en_d:
        v1 = fr_aligned[w]
        v2 = vecs_en[word]
        scores.append(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
        
    best_w = np.argsort(scores)[::-1][:5]
    print(np.array(en_d)[best_w])
        
for w in ['four','sun']:
    scores =[]
    for word in list(fr_aligned.keys()):
        v2 = fr_aligned[word]
        v1 = vecs_en[w]
        scores.append(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    best_w = np.argsort(scores)[::-1][:5]
    print(np.array(fr_d)[best_w])

['dog' 'poodle' 'terrier' 'dogs' 'spaniel']
['rope' 'door' 'hanger' 'stretcher' 'rack']
['quatre' 'six' 'trois' 'huit' 'cinq']
['sun' 'soleil' 'moon' 'astre' 'yun']


If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [8]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)




# TYPE CODE HERE

PATH_TO_DATA = "data/SST/"


with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.train'), encoding='utf-8') as f:
    train = f.read().splitlines()
    _train = [sent[:-3].split(" ") for sent in train]


    
with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.test.X'), encoding='utf-8') as f:
    test = f.read().splitlines()
    _test = [sent[:-3].split(" ") for sent in test]


    
with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.dev'), encoding='utf-8') as f:
    dev = f.read().splitlines()
    _dev = [sent[:-3].split(" ") for sent in dev]






In [32]:
# 2 - Encode sentences with the BoV model above

# TYPE CODE HERE

from sklearn.linear_model import LogisticRegression



s2v = BoV(w2v)
#idf = s2v.build_idf(_train)

idf = s2v.build_idf(_train)




encoded_tr = s2v.encode(_train,idf)

idf = s2v.build_idf(_test)


encoded_test = s2v.encode(_test)

idf = s2v.build_idf(_dev)


encoded_dev = s2v.encode(_dev)





In [35]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)

# TYPE CODE HERE

y = [x[0] for x in _train]
X = encoded_tr

dev_y = [x[0] for x in _dev]
dev_X = encoded_dev

clf = LogisticRegression(C = 12).fit(X, y)

alphas = [0.1,1,10]
for alpha in alphas:
    # instantiate and fit model
    clf = LogisticRegression(C = alpha).fit(X, y)
    # calculate errors
    score = clf.score(dev_X, dev_y)
    # print errors as report
    print('score at',alpha,'is',score)



/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


score at 0.01 is 0.5004541326067211


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


score at 0.1 is 0.6421435059037239


In [244]:
len(list(clf.predict(encoded_test)))

2210

In [263]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.



f = open("logreg_bov_y_test_sst.txt", "w")
for x in list(clf.predict(encoded_test)):
    f.write(str(x)+'\n')
f.close()


# TYPE CODE HERE

In [9]:
# BONUS!
# 5 - Try to improve performance with another classifier
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)

# TYPE CODE HERE

# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [38]:
import keras

In [39]:
# 1 - Load train/dev/test sets of SST
PATH_TO_DATA = "../../data/"

# TYPE CODE HERE
PATH_TO_DATA = "data/SST/"


with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.train'), encoding='utf-8') as f:
    train4 = f.read()
    train4_split = [x[2:-3] for x in train4.splitlines()]
    y_train = [int(x[0]) for x in train4.splitlines()]

    
with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.test.X'), encoding='utf-8') as f:
    test4 = f.read()
    test4_split = [x[:-3] for x in test4.splitlines()]


    
with io.open(os.path.join(PATH_TO_DATA, 'stsa.fine.dev'), encoding='utf-8') as f:
    dev4 = f.read()
    dev4_split = [x[2:-3] for x in dev4.splitlines()]
    y_dev = [int(x[0]) for x in dev4.splitlines()]


In [40]:
# 2 - Transform text to integers using keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/

# TYPE CODE HERE
#encode

from keras.preprocessing.text import text_to_word_sequence
# tokenize the document
result = text_to_word_sequence(train4+test4+dev4)
#compute the size of the dictionary
dict_size = len(set(result))*1.25



enc_train = [keras.preprocessing.text.one_hot(sent, dict_size) for sent in train4_split]

enc_dev = [keras.preprocessing.text.one_hot(sent, dict_size) for sent in dev4_split]

enc_test = [keras.preprocessing.text.one_hot(sent, dict_size) for sent in test4_split]




**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [41]:
# 3 - Pad your sequences using keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/

X_train=keras.preprocessing.sequence.pad_sequences(enc_train)

X_dev=keras.preprocessing.sequence.pad_sequences(enc_dev)

X_test=keras.preprocessing.sequence.pad_sequences(enc_test)




# TYPE CODE HERE


## 4.2 - Design and train your model

In [42]:
# 4 - Design your encoder + classifier using keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this contained : the lookuptable, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.


# ADAPT CODE BELOW


from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation

embed_dim  = 32  # word embedding dimension
nhid       = 64  # number of hidden units in the LSTM
vocab_size = 22300  # size of the vocabulary
n_classes  = 5

model = Sequential()
model.add(Embedding(vocab_size, embed_dim))
model.add(LSTM(nhid, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(n_classes, activation='sigmoid'))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`


In [43]:
# 5 - Define your loss/optimizer/metrics

# MODIFY CODE BELOW

loss_classif     =  'categorical_crossentropy' # find the right loss for multi-class classification
optimizer        =  'adam' # find the right optimizer
metrics_classif  =  ['accuracy']

# Observe how easy (but blackboxed) this is in Keras
model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          713600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 738,757
Trainable params: 738,757
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
# 6 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set

y_train_categorical = keras.utils.np_utils.to_categorical(y_train,5)
y_dev_categorical = keras.utils.np_utils.to_categorical(y_dev,5)

# ADAPT CODE BELOW
bs = 64
n_epochs = 6

history = model.fit(X_train, y_train_categorical, batch_size=bs, nb_epoch=n_epochs, validation_data=(X_dev, y_dev_categorical))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 8544 samples, validate on 1101 samples
Epoch 1/6
8544/8544 [==============================] - 14s 2ms/step - loss: 1.5754 - acc: 0.2707 - val_loss: 1.5719 - val_acc: 0.2534
Epoch 2/6
8544/8544 [==============================] - 13s 2ms/step - loss: 1.5345 - acc: 0.3095 - val_loss: 1.4928 - val_acc: 0.3460
Epoch 3/6
8544/8544 [==============================] - 13s 2ms/step - loss: 1.3161 - acc: 0.4320 - val_loss: 1.3983 - val_acc: 0.3869
Epoch 4/6
8544/8544 [==============================] - 13s 2ms/step - loss: 1.0561 - acc: 0.5520 - val_loss: 1.4944 - val_acc: 0.3860
Epoch 5/6
8544/8544 [==============================] - 13s 2ms/step - loss: 0.7925 - acc: 0.7090 - val_loss: 1.7135 - val_acc: 0.3342
Epoch 6/6
8544/8544 [==============================] - 13s 2ms/step - loss: 0.5860 - acc: 0.7925 - val_loss: 1.9602 - val_acc: 0.3515


In [390]:
scores = model.predict_classes(X_test)


In [393]:
# 7 - Generate your predictions on the test set using model.predict(x_test)
#     https://keras.io/models/model/
#     Log your predictions in a file (one line = one integer: 0,1,2,3,4)
#     Attach the output file "logreg_lstm_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE
scores = list(model.predict_classes(X_test))

f = open("logreg_lstm_y_test_sst.txt", "w")
for x in list(scores):
    f.write(str(x)+'\n')
f.close()

## 4.3 -- innovate !

In [ ]:
# 8 - Open question: find a model that is better on your dev set
#     (e.g: use a 1D ConvNet, use a better classifier, pretrain your lookup tables ..)
#     you will get point if the results on the test set are better: be careful of not overfitting your dev set too much..
#     Attach the output file "XXX_XXX_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE
